# Artificial and Computational Intelligence Assignment 2

## Problem solving Using Constraint Satisfaction

Things to follow
1.	Use appropriate data structures to represent the graph using python libraries
2.	Provide proper documentation
3.	Find the path and print it

## **Coding begins here for CSP**

### 1.Implement Python code for the design under part a, using corresponding algorithm given in the problem statement.

In [385]:
#Code Block : Function for algorithm implementation
# Base class for all constraints
from typing import Generic, TypeVar, Dict, List, Optional
from abc import ABC, abstractmethod

V = TypeVar('V') # variable type
D = TypeVar('D') # domain type

class Constraint(Generic[V, D], ABC):
    # The variables that the constraint is between
    def __init__(self, variables: List[V]):
        self.variables = variables


    # Must be overridden by subclasses
    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]):
        ...
# A CSP(constraint satisfaction problem) consists of variables of type V
# that have ranges of values known as domains of type D and constraints
# that determine whether a particular variable's domain selection is valid
class CSPBackTracking(Generic[V, D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]):
        self.variables: List[V] = variables # variables to be constrained
        self.domains: Dict[V, List[D]] = domains # domain of each variable
        self.constraints: Dict[V, List[Constraint[V, D]]] = {}
        self.curr_domains=None
        self.neighbors = self.variables
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("Every variable should have a domainassigned to it.")

    def add_constraint(self, constraint: Constraint[V, D]):
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("Variable in constraint not in CSP")
            else:
                self.constraints[variable].append(constraint)


    # Check if the value assignment is consistent by checking all constraints
    # for the given variable against it
    def consistent(self, variable: V, assignment: Dict[V, D]):
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
        return True

    def backtracking_search(self, assignment: Dict[V, D] = {}):
        # assignment is complete if every variable is assigned (our base case)
        if len(assignment) == len(self.variables):
            return assignment

        # get all variables in the CSP but not in the assignment
        unassigned: List[V] = [v for v in self.variables if v not in assignment]
        len_of_domains = len(self.domains)
        # get the every possible domain value of the first unassigned variable
        first: V = unassigned[0]
        for value in self.domains[first]:
            local_assignment = assignment.copy()
            if(first == len_of_domains):
                local_assignment[first] = self.domains[first][len(self.domains[first])-1]
            else:
                local_assignment[first] = value
            # if we're still consistent, we recurse (continue)
            if self.consistent(first, local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
                # if we didn't find the result, we will end up backtracking
                if result is not None:
                    return result
        return None 
    
    
class ScheduleConstraint(Constraint[int, int]):
    def __init__(self, columns: List[int]):
        super().__init__(columns)
        self.columns: List[int] = columns

    def satisfied(self, assignment: Dict[int, int]):
       # q1c = column 1 , q1r = row 1 
        for q1c, q1r in assignment.items(): 
        # q2c = column 2 
            for q2c in range(q1c+1, len(self.columns)): 
                if q2c in assignment:
                    q2r: int = assignment[q2c] 
                    if q1r == q2r: 
                        return False
                    
        return True # no conflict


In [386]:
### 2.	Heuristics function 1 as representented in problem statement 
# Base class for all constraints
from typing import Generic, TypeVar, Dict, List, Optional
from abc import ABC, abstractmethod

V = TypeVar('V') # variable type
D = TypeVar('D') # domain type

class Constraint(Generic[V, D], ABC):
    # The variables that the constraint is between
    def __init__(self, variables: List[V]):
        self.variables = variables


    # Must be overridden by subclasses
    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]):
        ...
class CSPHeuristicsSearch(Generic[V, D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]):
        self.variables: List[V] = variables # variables to be constrained
        self.domains: Dict[V, List[D]] = domains # domain of each variable
        self.constraints: Dict[V, List[Constraint[V, D]]] = {}
        self.curr_domains=None
        self.neighbors = self.variables
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("Every variable should have a domainassigned to it.")

    def add_constraint(self, constraint: Constraint[V, D]):
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("Variable in constraint not in CSP")
            else:
                self.constraints[variable].append(constraint)


    # Check if the value assignment is consistent by checking all constraints
    # for the given variable against it
    def consistent(self, variable: V, assignment: Dict[V, D]):
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
        return True
    
    def recursive_backtracking(self,assignment: Dict[V, D] = {}):
        if len(assignment) == len(self.variables):
            return assignment
        var = self.mrv(assignment, csp)
        for value in self.mcv(var, assignment, csp):
            if 1 == self.nconflicts(var, value, assignment):
                self.assign(var, value, assignment)
                removals = self.start(var, value)
                result = self.recursive_backtracking(assignment)
                if result is not None:
                    return result
                self.undo_val(removals)
        self.unassign(var, assignment)
        return None

# Most Remaining Values (MRV) Heuristic:
    def mrv(self, assignment, csp):

        min= self.select_unassigned_variable( assignment, csp)
        for v in self.variables:
            if v not in assignment:
                if self.curr_domains:
                    r= len(self.curr_domains[v])
                    n= len(self.curr_domains[min])
                else:
                    count=0
                    for val in self.domains[v]:
                        if self.nconflicts(v, val, assignment) == 0:
                            count=count+1
                    n=count
                    count = 0
                    for val in self.domains[min]:
                        if self.nconflicts(min, val, assignment) == 0:
                            count = count + 1
                    r = count
                if r < n:
                    min = v
        return min
    
    def heuristics_search(self,assignment: Dict[V, D] = {}):
        self.domains = dict((v, list(reversed(self.domains[v])))
                                     for v in self.variables)
        return self.recursive_backtracking(assignment)
        
    
### Heuristics function 2 as representented in problem statement
#Choose the variable that imposes the most constraints on the remaining variables

    def mcv(self,var, assignment, csp):

        return sorted(self.left_domain(var),
                      key=lambda val: self.consistent(var, assignment))

    def left_domain(self, var):
        return (self.curr_domains or self.domains)[var]
    
    def select_unassigned_variable(self,assignment, csp):
        for v in self.variables:
            if v not in assignment:
                return v
            
    def nconflicts(self, var, val, assignment):
        if self.consistent(var, assignment):
            return 1
        return 0
    
    def support_pruning(self):
        if self.curr_domains is None:
            self.curr_domains = dict((v, list(self.domains[v]))
                                     for v in self.variables)

    def start(self, var, value):
        self.support_pruning()
        removals = [(var, d) for d in self.curr_domains[var] if d != value]
        self.curr_domains[var] = [value]
        return removals
    
    def undo_val(self, removals):
        for B, b in removals:
            self.curr_domains[B].append(b)
    
    def assign(self, var, val, assignment):
        assignment[var] = val
        
    def unassign(self, var, assignment):
        if var in assignment:
            del assignment[var]


### DYNAMIC INPUT

In [387]:
#IMPORTANT : Dynamic Input must be got in this section. This is applicable for all the relevent problems as mentioned in the question. 
import ast
#Input is available as part of CSV files i.e.
while True:
    try:
        inputFile = "Input_Entry.txt"
        # Declare an empty array
        arr = []
        lines = open(inputFile, 'r').readlines()[1:]
        arrayIndex = 0
        # Throw error if input file does not have any entries
        if len(lines) ==0:
            raise ValueError ("Input file missing entries")
        # Remove empty lines
        lines = filter(lambda x: x.strip(), lines)
        for line in lines:
            values = [str(e) for e in line.rstrip("\n").split(',')]
            arr.insert(arrayIndex,values)
            arrayIndex = arrayIndex + 1
    except FileNotFoundError:
        print("Wrong file or file path, please try again!")
    else:
        break
            


In [388]:
#Code Block : Function & call to get inputs (start/end state)
# Input section
if __name__ == "__main__":
    number_of_flights = len(arr[0])
    flight_of_serial_numbers = list(range(1, number_of_flights+1))
    flight_dictionary[int,str] = dict(zip(flight_of_serial_numbers, arr[0]))
    rows: List[int] = list(range(1, number_of_flights+1))
    columns: Dict[int, List[str]] = {}
    for column in rows:
        columns[column] = arr[1]
    
    if(len(rows) > len(columns[1]) and (len(rows) - len(columns[1]) !=1)):
        print("No solution found! Possibly insufficient available slots. Please add more slots and check")
    csp: CSPBackTracking[int, int] = CSPBackTracking(rows,columns)
    csp.add_constraint(ScheduleConstraint(rows))
    solution: Optional[Dict[int, str]] = csp.backtracking_search()
    print("Backtracking Result:",solution)
    
    # Heuristics search including MCV and MRV
    csp: CSPHeuristicsSearch[int, int] = CSPHeuristicsSearch(rows,columns)
    csp.add_constraint(ScheduleConstraint(rows))
    solution: Optional[Dict[int, str]] = csp.heuristics_search()
    if solution is None:
        print("No solution found!")
    else:
        print("Heuristics including MCV and MRV:",solution)

Backtracking Result: {1: 'SixAM_Slot', 2: 'SevenAM_Slot', 3: 'EightAM_Slot', 4: 'NineAM_Slot', 5: 'NineAM_Slot'}
Heuristics including MCV and MRV: {1: 'SixAM_Slot', 2: 'SevenAM_Slot', 3: 'EightAM_Slot', 4: 'NineAM_Slot', 5: 'NineAM_Slot'}


### 3.Print the final result as per the given problem statement

### DFS based Backtracking Result for the given problem:
{'J-9W665: 6:00 AM IST', 'J-9W601 : 7:00 AM IST', 'J-9W4331: 8:00 AM IST', 'J-9W4569: 9:00 AM IST', 'J-9W377: 9:00 AM IST'}
### MCV and MRV Heuristic Result for the given problem:
{'J-9W665: 6:00 AM IST', 'J-9W601 : 7:00 AM IST', 'J-9W4331: 8:00 AM IST', 'J-9W4569: 9:00 AM IST', 'J-9W377: 9:00 AM IST'}


### Code Block : Print the Time & Space complexity of algorithm 1 

A complete CSP depth search based back tracking search algorithm that returns a solution if one exists or
report insolubility otherwise have exponential time-complexity. In regards to the complexity of backtracking algorithms, 
we follow two agreements:
    
– We express the time-complexity (upper bound) by O∗ notation which suppresses the polynomial factor. 

– We express the time complexity of a CSP search algorithm by the number of partial solutions generated by 
the algorithm where successor states are generated by considering assignments to a single variable. 

With n variables and maximum domain size d is widely considered to be O∗(d^n). Therefore, it is O*(4^5) = O(1024)

To store the domains of the problem requires O(na) space. The algorithm does not require more temporary memory 
than O(n) to store the compound label. Therefore, the space complexity of Chronological_Backtracking is O(nd) = O(20).

### Code Block : Print the Time & Space complexity of algorithm 2

we found that a combination of heuristic methods of constraint solving can reduce the time complexity. 
In particular, we prove that the MCV-MRV algorithm combined with the fail-first variable ordering heuristic (FF) 
achieves time complexity of O *((d – 1)n ), where n and d are the number of variables and the maximal domain size 
of the given CSP, respectively. Therefore, it is O*(3*5) = O(15) which is also space complexity.

Furthermore, we found that the combination is essential because neither MCV-MRV alone nor MRV with FF achieve 
the above complexity. 